In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

this notebook contains three models namely
 
1.   DecisionTree Regression(using Grid search hyperparametr tunning)

2.    RandomForest Regression(using Random search hyperparameter tunning)

1.   Linear Regression model

The models were evaluated using the following evaluation metrics:

1. Mean absolute error(MAE)
2. Root mean square error (RMSE)

1. R-squared value.

the initial parts of the notebook contains the codes for loading the datasets, splitting the dataset into test and train sets.

subsequently the models were trained in succession before the metrics were evaluated at the bottom of the notebook to check for the best performing model.

the model with the highest R-squared value and the lowest RMSE value is chosen as the best performing model.

**Loading Dataset**

In [2]:
data_set = pd.read_csv('https://raw.githubusercontent.com/HamoyeHQ/01-University-Admissions/master/data/Admission_Predict.csv')
data_set.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In [3]:
data_set.set_index('Serial No.',inplace = True)
data_set.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,324,107,4,4.0,4.5,8.87,1,0.76
3,316,104,3,3.0,3.5,8.00,1,0.72
4,322,110,3,3.5,2.5,8.67,1,0.80
5,314,103,2,2.0,3.0,8.21,0,0.65


**Defining dependent and Independent Variables**

In [4]:
x = data_set.drop(['Chance of Admit '], axis = 1)
y = data_set['Chance of Admit ']

In [5]:
x.head(3)

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
Serial No.,,,,,,,
1,337,118,4,4.5,4.5,9.65,1
2,324,107,4,4.0,4.5,8.87,1
3,316,104,3,3.0,3.5,8.00,1


In [6]:
y.head(3)

Serial No.
1    0.92
2    0.76
3    0.72
Name: Chance of Admit , dtype: float64

**Splitting Data to Train and Test Split**

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(x, y, test_size = 0.25, random_state = 0)

In [8]:
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(300, 7)
(100, 7)
(300,)
(100,)


In [9]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_train = scaler.fit_transform(X_train)
scaled_val = scaler.transform(X_val)


**Decision Tree**

grid search hyperparameter tunning for decision tree

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
tree = DecisionTreeRegressor()
parameters = {'criterion':('mse','mae'),'max_features':('auto','sqrt','log2'),
              'min_samples_split':[2,3,5,7,9],'min_samples_leaf': [1,2,4,6,8]}
grid_search = GridSearchCV(tree,parameters,scoring='neg_mean_absolute_error')
grid_search.fit(scaled_train,y_train)
                      

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse',
                                             max_depth=None, max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort='deprecated',
                                             random_state=None,
                                             splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'criterion': ('mse', 'mae'),
                         'max_features': ('auto', 'sqrt', 'log2'),
                         'min_s

In [11]:
print(grid_search.best_score_)
print(grid_search.best_estimator_)
grid_search.best_params_

-0.04968333333333333
DecisionTreeRegressor(ccp_alpha=0.0, criterion='mae', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=8, min_samples_split=9,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')


{'criterion': 'mae',
 'max_features': 'auto',
 'min_samples_leaf': 8,
 'min_samples_split': 9}

In [12]:
tree = grid_search.best_estimator_ #training decision tree with best parameters from search
tree.fit(scaled_train,y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mae', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=8, min_samples_split=9,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [13]:
tree_pred = tree.predict(scaled_val)

**Random Forest**

Random Search hyperparameter Tunning for RandomForest regression

In [14]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor()
n_estimators = range(10,300,10)
criterion = ('mse','mae')
min_samples_split = [2,3,5,7,9] #minimum number of splits
min_samples_leaf = [1,2,4,6,8] #minimum number of samples for leaf node
max_features = ['auto','sqrt','log2',None] #the number of features to consider when looking for the best split
hyperparameter_grid = {'min_samples_leaf':min_samples_leaf,
                       'min_samples_split':min_samples_split,
                       'max_features':max_features,
                       'criterion': criterion,
                       'n_estimators': n_estimators}
                       

In [15]:
from sklearn.model_selection import RandomizedSearchCV
random_search = RandomizedSearchCV(estimator = forest,
                                   param_distributions=hyperparameter_grid,scoring = 'neg_mean_absolute_error', cv = 5)
random_search = random_search.fit(scaled_train,y_train)

In [16]:
print(random_search.best_score_)
print(random_search.best_estimator_)
random_search.best_params_

-0.04585493055555561
RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mae',
                      max_depth=None, max_features='log2', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=2,
                      min_samples_split=7, min_weight_fraction_leaf=0.0,
                      n_estimators=240, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)


{'criterion': 'mae',
 'max_features': 'log2',
 'min_samples_leaf': 2,
 'min_samples_split': 7,
 'n_estimators': 240}

In [17]:
forest = random_search.best_estimator_ #training random forest with best estimator from random search
forest.fit(scaled_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mae',
                      max_depth=None, max_features='log2', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=2,
                      min_samples_split=7, min_weight_fraction_leaf=0.0,
                      n_estimators=240, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [18]:
forest_pred = forest.predict(scaled_val)

**Linear Regression Model**

In [19]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(scaled_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [20]:
linear_pred = regressor.predict(scaled_val)

**r2_score**

In [21]:
from sklearn.metrics import r2_score
#R-squared
print(f'the R-squared value of the DecisionTree model is : {round(r2_score(y_val,tree_pred),4)}')
print(f'the R-squared value of the Random Forest model is : {round(r2_score(y_val,forest_pred),4)}')
print(f'the R-squared value of the Linear Regression model is : {round(r2_score(y_val,linear_pred),4)}')

the R-squared value of the DecisionTree model is : 0.5878
the R-squared value of the Random Forest model is : 0.7213
the R-squared value of the Linear Regression model is : 0.7249


**mean absolute error**

In [22]:
from sklearn.metrics import mean_absolute_error as mae
#MAE
print("the mean absolute error of the DecisionTree model is: {}".format(round(mae(y_val,tree_pred),4)))
print("the mean absolute error of the Random Forest model is: {}".format(round(mae(y_val,forest_pred),4)))
print("the mean absolute error of the Linear model is: {}".format(round(mae(y_val,linear_pred),4)))

the mean absolute error of the DecisionTree model is: 0.06
the mean absolute error of the Random Forest model is: 0.0472
the mean absolute error of the Linear model is: 0.0491


**root mean square error**

In [23]:
from sklearn.metrics import mean_squared_error as mse
#RMSE
print("ther root mean square error of the DecisionTree Model is: %s" %round(np.sqrt(mse(y_val,tree_pred)),4))
print("ther root mean square error of the Random Forest Model is: %s" %round(np.sqrt(mse(y_val,forest_pred)),4))
print("ther root mean square error of the Linear Model is: %s" %round(np.sqrt(mse(y_val,linear_pred)),4))


ther root mean square error of the DecisionTree Model is: 0.0848
ther root mean square error of the Random Forest Model is: 0.0697
ther root mean square error of the Linear Model is: 0.0693
